In [1]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import OllamaEmbeddings

## 📦 Imports e Configuração

Bibliotecas necessárias:
- `os`, `Path`, `load_dotenv`: Gerenciamento de arquivos e variáveis de ambiente
- `FAISS`: Banco de dados vetorial (com suporte a salvar/carregar!)
- `OllamaEmbeddings`: Embeddings locais via Ollama

### Por que Ollama?

Neste lab usamos Ollama porque:
- ✅ **100% Gratuito** - Nenhum custo de API
- ✅ **Privacidade Total** - Dados não saem da sua máquina
- ✅ **Offline** - Funciona sem internet
- ✅ **Performance** - Roda local, sem latência de rede
- ✅ **Modelos variados** - all-minilm, nomic-embed-text, mxbai-embed-large

### Pré-requisitos

1. **Docker Compose** deve estar rodando:
   ```bash
   docker-compose up -d
   ```

2. **Modelos instalados** (já incluídos no `Dockerfile.ollama`):
   - `all-minilm` (embeddings - 384 dims)
   - `nomic-embed-text` (embeddings - 768 dims)
   - `llama3.2:1b` (LLM para geração)

3. **Ollama acessível** em `http://ollama:11434` (dentro do Docker) ou `http://localhost:11434`

In [2]:
# Configuração do endpoint Ollama
# Se estiver rodando fora do Docker, use: http://localhost:11434
# Se estiver dentro do Docker (Jupyter no compose), use: http://ollama:11434
OLLAMA_BASE_URL = 'http://localhost:11434'
# OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434')

print(f'🔗 Ollama endpoint: {OLLAMA_BASE_URL}')

# Testar conexão
try:
    response = requests.get(f'{OLLAMA_BASE_URL}/api/tags')
    if response.status_code == 200:
        models = response.json().get('models', [])
        print(f'✅ Ollama conectado! Modelos disponíveis: {len(models)}')
        for model in models:
            print(f"   - {model['name']}")
    else:
        print('⚠️  Ollama respondeu, mas com erro')
except Exception as e:
    print(f'❌ Erro ao conectar com Ollama: {e}')
    print('   Certifique-se que o Docker Compose está rodando!')

🔗 Ollama endpoint: http://localhost:11434
✅ Ollama conectado! Modelos disponíveis: 6
   - embeddinggemma:latest
   - gpt-oss:latest
   - llama3.2:1b
   - all-minilm:latest
   - mxbai-embed-large:latest
   - nomic-embed-text:latest


## 🔐 Verificando Conexão com Ollama

Antes de começar, precisamos garantir que o Ollama está acessível.

**Troubleshooting:**

Se você ver ❌, verifique:
1. Docker Compose está rodando? `docker-compose ps`
2. Container Ollama está UP? `docker-compose logs ollama`
3. Porta 11434 está exposta? Verifique o `docker-compose.yaml`

**Endpoints:**
- Dentro do Docker: `http://ollama:11434`
- Fora do Docker: `http://localhost:11434`

In [4]:
meus_textos = [
    "O novo iPhone 15 tem uma lente periscópica incrível.",    # Tecnologia
    "Para fazer um bolo macio, bata as claras em neve.",       # Culinária
    "O atacante chutou a bola no ângulo e foi gol.",           # Esporte
    "A placa de vídeo RTX 4090 roda jogos em 4K.",             # Tecnologia
    "Receita de lasanha à bolonhesa com muito queijo."         # Culinária
]

## 📄 Passo 1: Preparando os Documentos

Nosso dataset de teste com 5 documentos de categorias diferentes.

**Importante:** Em uma aplicação RAG real, esses textos viriam de:
- 📑 PDFs de documentação
- 🌐 Base de conhecimento da empresa
- 💬 FAQs e tickets de suporte
- 📊 Manuais técnicos

💡 **Conceito RAG:** Quanto mais documentos relevantes você indexar, melhor o LLM conseguirá responder perguntas específicas do seu domínio!

In [5]:
# Escolha o modelo de embeddings
# Opções: 'all-minilm' (384 dims, rápido), 'nomic-embed-text' (768 dims, balanceado), 'mxbai-embed-large' (1024 dims, preciso)
EMBEDDING_MODEL = 'mxbai-embed-large'

embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=OLLAMA_BASE_URL
)

print(f'✅ Modelo de embeddings inicializado: {EMBEDDING_MODEL}')

✅ Modelo de embeddings inicializado: mxbai-embed-large


C:\Users\Max\AppData\Local\Temp\ipykernel_46504\4109340853.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


## 🧠 Passo 2: Inicializando o Modelo de Embeddings

Criamos uma instância do modelo de embeddings via Ollama local.

**Modelo:** `nomic-embed-text` (recomendado)
- **Dimensões:** 768 (excelente qualidade semântica)
- **Qualidade:** ⭐⭐⭐⭐⭐ Estado da arte open-source
- **Custo:** $0.00 (totalmente gratuito!)
- **Velocidade:** ~50-100ms por request (local)
- **Privacidade:** 🔒 100% local, dados não saem da máquina

**Alternativas:**
- `all-minilm`: 384 dims, mais rápido, menor qualidade
- `mxbai-embed-large`: 1024 dims, melhor qualidade, mais lento

### Como funciona?

```
Texto → 🏠 Ollama Local → Vetor [768 números]
```

Cada texto será transformado em um vetor de 768 dimensões que captura seu significado semântico, **sem enviar dados para nuvem**!

In [6]:
print('⏳ Criando índice FAISS... (pode levar alguns segundos)')
vector_store = FAISS.from_texts(meus_textos, embeddings)
print('✅ Índice criado com sucesso!')

⏳ Criando índice FAISS... (pode levar alguns segundos)
✅ Índice criado com sucesso!
✅ Índice criado com sucesso!


## 🗄️ Passo 3: Criando o Índice Vetorial

Transformamos os textos em vetores e criamos o índice FAISS **em memória**.

### O que acontece internamente?

```text
Para cada texto em meus_textos:
  1. Envia para Ollama local (http://localhost:11434)
  2. Recebe vetor de 768 dimensões
  3. FAISS armazena o vetor em RAM

Resultado:
  vector_store = Índice FAISS com 5 vetores em memória
```

**Importante:** Neste ponto, o índice existe **apenas na RAM**! Se você fechar o programa, perde tudo.

⏱️ **Tempo:** ~0.5-1 segundo (5 requests locais ao Ollama)

💡 **Vantagem Ollama:** Como é local, é **muito mais rápido** que APIs de nuvem (sem latência de rede)!

In [7]:
FAISS_PATH = Path.cwd().joinpath('..', '..', 'data', 'meu_indice_faiss_ollama')

vector_store.save_local(str(FAISS_PATH))
print(f"✅ Banco vetorial salvo com sucesso em: {FAISS_PATH}")

✅ Banco vetorial salvo com sucesso em: e:\01-projetos\11-work\11.34-engenharia-vetorial\src\1_fundamentos\..\..\data\meu_indice_faiss_ollama


## 💾 Passo 4: Salvando o Índice no Disco (PERSISTÊNCIA!)

Esta é a **parte mais importante** para aplicações RAG em produção!

### O que acontece aqui?

```python
vector_store.save_local(str(FAISS_PATH))
```

**Internamente:**
1. FAISS cria uma pasta: `data/meu_indice_faiss_ollama/`
2. Salva 2 arquivos:
   - `index.faiss` → Os vetores e estrutura de índice
   - `index.pkl` → Metadados (textos originais, etc.)

### Estrutura de arquivos criada:

```text
data/
└── meu_indice_faiss_ollama/
    ├── index.faiss  ← ~60KB (5 vetores × 768 dims × 4 bytes)
    └── index.pkl    ← ~2KB (textos originais)
```

### Por que isso é importante?

**Sem persistência:**
```python
# Toda execução:
vector_store = FAISS.from_texts(textos, embeddings)
# ↑ 5 requests ao Ollama + 0.5-1 segundo
```

**Com persistência:**
```python
# Apenas uma vez:
vector_store.save_local("path")

# Depois, sempre:
vector_store = FAISS.load_local("path", embeddings)
# ↑ 0 requests ao Ollama + ~10ms
```

### Benefícios em produção

| Métrica | Sem Persistência | Com Persistência |
|---------|------------------|------------------|
| **Tempo de startup** | Segundos/Minutos | Milissegundos |
| **Carga no Ollama** | Toda vez | Uma vez (na criação) |
| **Escalabilidade** | Limitada | Milhões de docs |
| **Confiabilidade** | Dependente do Ollama | Funciona offline |

💡 **Analogia:** É como salvar um documento do Word - você não quer redigitar tudo cada vez que abrir!

In [8]:
# CARREGAR do disco
# O parâmetro 'allow_dangerous_deserialization' é necessário em versões recentes
# para confirmar que você confia no arquivo que está carregando.
print('⏳ Carregando índice do disco...')
novo_db = FAISS.load_local(
    str(FAISS_PATH), 
    embeddings, 
    allow_dangerous_deserialization=True
)
print('✅ Índice carregado com sucesso!')

⏳ Carregando índice do disco...
✅ Índice carregado com sucesso!


In [9]:
# Visualizar os textos carregados
print('📚 Documentos no índice:')
for doc_id in novo_db.index_to_docstore_id.values():
    documento = novo_db.docstore.search(doc_id)
    print(f'  • {documento.page_content}')

📚 Documentos no índice:
  • O novo iPhone 15 tem uma lente periscópica incrível.
  • Para fazer um bolo macio, bata as claras em neve.
  • O atacante chutou a bola no ângulo e foi gol.
  • A placa de vídeo RTX 4090 roda jogos em 4K.
  • Receita de lasanha à bolonhesa com muito queijo.


## 📂 Passo 5: Carregando o Índice do Disco

Agora vamos **carregar** o índice que salvamos anteriormente!

### O que acontece aqui?

```python
novo_db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)
```

**Internamente:**
1. FAISS lê os arquivos da pasta `data/meu_indice_faiss_ollama/`
2. Reconstrói o índice em memória (~10ms)
3. Pronto para uso instantâneo!

### Parâmetros importantes

**`embeddings`**: Necessário para futuras queries
- Não recalcula nada dos documentos existentes
- Só será usado quando você buscar algo novo

**`allow_dangerous_deserialization=True`**: Confirmação de segurança
- FAISS usa pickle para serializar
- Pickle pode executar código malicioso se vier de fonte não confiável
- Como você mesmo criou o índice, é seguro ✅

### Fluxo completo de uma aplicação RAG

```text
Primeira execução (setup):
  1. Carregar documentos
  2. Criar embeddings (via Ollama)
  3. Criar índice FAISS
  4. Salvar no disco ← Você está aqui!

Execuções subsequentes (produção):
  5. Carregar índice do disco ← Estamos fazendo isso agora!
  6. Receber query do usuário
  7. Buscar documentos relevantes
  8. Enviar contexto + query para LLM
  9. Retornar resposta
```

### Performance com Ollama

| Operação | Tempo (5 docs) | Tempo (10k docs) | Tempo (1M docs) |
|----------|----------------|------------------|------------------|
| **Criar índice** | ~1s | ~15s | ~15min |
| **Salvar** | ~50ms | ~200ms | ~5s |
| **Carregar** | ~10ms | ~100ms | ~2s |
| **Buscar** | <1ms | ~5ms | ~50ms |

💡 **Observe:** Carregar é **muito mais rápido** que criar! E com Ollama local, é ainda mais rápido que APIs de nuvem!

In [10]:
# Usar normalmente
consulta = "Sugestão de celular"
print(f'🔍 Buscando por: "{consulta}"')
resultados = novo_db.similarity_search(consulta, k=2)
print(f'✅ Encontrados {len(resultados)} documentos relevantes')

🔍 Buscando por: "Sugestão de celular"
✅ Encontrados 2 documentos relevantes
✅ Encontrados 2 documentos relevantes


## 🔍 Passo 6: Usando o Índice Carregado

Agora vamos fazer uma busca usando o índice que **carregamos do disco**!

### A Query

```python
consulta = "Sugestão de celular"
```

**Desafio semântico:**
- A palavra "iPhone" não aparece na query
- A palavra "celular" sim
- O modelo precisa entender que iPhone é um celular

### O que acontece internamente?

1. **Query vira vetor (via Ollama local):**
   ```text
   "Sugestão de celular" → 🏠 Ollama → [0.15, -0.32, ..., 0.47]
   ```

2. **FAISS busca (local, instantâneo):**
   ```text
   Compara o vetor da query com os 5 vetores armazenados
   Calcula distâncias:
     - iPhone: 0.3 ← Mais próximo!
     - RTX 4090: 1.2
     - Bolo: 2.5
     - Gol: 2.8
     - Lasanha: 2.6
   ```

3. **Retorna top-k (k=2):**
   ```
   [iPhone, RTX 4090]
   ```

### Parâmetro k

- `k=1`: Retorna apenas o mais similar
- `k=2`: Retorna os 2 mais similares
- `k=5`: Retorna todos (nosso dataset tem 5)

💡 **Dica:** Em RAG, geralmente usa-se `k=3` ou `k=5` para dar contexto suficiente ao LLM sem sobrecarregar.

In [11]:
print('📄 Documento mais relevante:')
print(f'   "{resultados[0].page_content}"')

📄 Documento mais relevante:
   "O novo iPhone 15 tem uma lente periscópica incrível."


## 📊 Passo 7: Visualizando o Resultado

Vamos ver o documento mais relevante (índice 0 = primeiro resultado).

**Esperado:** Deve retornar algo sobre iPhone, pois é o documento sobre celular!

### Interpretando o resultado

Se você ver:
- ✅ `"O novo iPhone 15..."` → Perfeito! A busca funcionou
- ❌ Qualquer outro texto → Algo deu errado (improvável com nomic-embed-text)

### Qualidade do modelo nomic-embed-text

O modelo `nomic-embed-text` é treinado especificamente para busca semântica e tem **excelente desempenho** em tarefas de similaridade, comparável a modelos comerciais!

### Próximo passo: RAG completo com Ollama

Em um RAG de verdade com Ollama, você faria:

```python
# 1. Buscar documentos relevantes (já fizemos!)
resultados = novo_db.similarity_search(consulta, k=3)

# 2. Montar o contexto
contexto = "\n".join([doc.page_content for doc in resultados])

# 3. Criar prompt para LLM
prompt = f"""
Baseado no seguinte contexto:
{contexto}

Responda a pergunta: {consulta}
"""

# 4. Enviar para LLM Ollama (Llama, Mistral, etc.)
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2:1b", base_url=OLLAMA_BASE_URL)
resposta = llm.invoke(prompt)

print(resposta)
# → "Eu recomendo o iPhone 15, que possui uma lente periscópica incrível..."
```

💡 **Esse é o poder do RAG:** O LLM responde com base nos **seus** documentos, não apenas no conhecimento geral! E tudo **100% local e gratuito**!

In [15]:
from langchain_community.llms import Ollama

def rag_query(pergunta, k=2, modelo='llama3.2:1b'):
    """
    Realiza uma busca RAG completa com Ollama local:
    1. Carrega o índice do disco
    2. Busca documentos relevantes
    3. Envia para o LLM local gerar a resposta
    """
    
    # 1. Carregar índice do disco (garantindo que usamos a versão persistida)
    db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)
    
    # 2. Buscar documentos relevantes (Retrieval)
    docs = db.similarity_search(pergunta, k=k)
    
    # 3. Montar o contexto (Augmentation)
    contexto = "\n".join([f"- {doc.page_content}" for doc in docs])
    
    # 4. Criar prompt
    prompt = f"""
Você é um assistente inteligente. Use APENAS o contexto abaixo para responder a pergunta do usuário.
Se a resposta não estiver no contexto, diga educadamente que não possui essa informação.

Contexto:
{contexto}

Pergunta: {pergunta}

Resposta:"""
    
    # 5. Gerar resposta (Generation) - usando Llama local
    llm = Ollama(
        model=modelo,  # Modelo leve e rápido
        base_url=OLLAMA_BASE_URL,
        temperature=0.1
    )
    resposta = llm.invoke(prompt)
    
    # Retorna a resposta (texto) e os documentos usados (fonte)
    return resposta, docs

print('✅ Função RAG completa criada!')

✅ Função RAG completa criada!


In [17]:
# Vamos testar nosso sistema RAG!
pergunta = "Qual celular tem uma câmera boa?"

print(f"🤖 Pergunta: {pergunta}\n")
print("⏳ Processando RAG local...\n")

resposta_final, docs_usados = rag_query(pergunta, modelo='gpt-oss:latest')

print("📝 Resposta do Llama (local):")
print("-" * 40)
print(resposta_final)
print("-" * 40)

print("\n\n📚 Documentos usados no contexto:")
for i, doc in enumerate(docs_usados):
    print(f"   {i+1}. {doc.page_content}")

🤖 Pergunta: Qual celular tem uma câmera boa?

⏳ Processando RAG local...

📝 Resposta do Llama (local):
----------------------------------------
O iPhone 15 tem uma câmera excelente, graças à sua lente periscópica incrível.
----------------------------------------


📚 Documentos usados no contexto:
   1. O novo iPhone 15 tem uma lente periscópica incrível.
   2. O atacante chutou a bola no ângulo e foi gol.
📝 Resposta do Llama (local):
----------------------------------------
O iPhone 15 tem uma câmera excelente, graças à sua lente periscópica incrível.
----------------------------------------


📚 Documentos usados no contexto:
   1. O novo iPhone 15 tem uma lente periscópica incrível.
   2. O atacante chutou a bola no ângulo e foi gol.


### 🧠 Entendendo a "Mágica"

Observe o que aconteceu acima:

1.  **Recuperação (Retrieval):** O sistema buscou no FAISS os trechos mais parecidos com "câmera boa". Ele encontrou reviews de celulares (ex: "O iPhone 15 tem uma lente periscópica incrível...").
2.  **Augmentação (Augmentation):** Pegamos esses trechos e colamos no prompt do Llama.
3.  **Geração (Generation):** O Llama leu os trechos e respondeu a pergunta **baseado apenas neles**.

É como se você desse um livro para alguém e dissesse: "Responda a pergunta usando APENAS este livro".

### 🚀 Vantagens do RAG com Ollama

✅ **100% Gratuito** - Sem custos de API  
✅ **Privacidade Total** - Dados não saem da máquina  
✅ **Offline** - Funciona sem internet  
✅ **Rápido** - Sem latência de rede  
✅ **Escalável** - Pode rodar em servidores próprios  

### 🧪 Experimento: Perguntas fora do contexto

O que acontece se perguntarmos algo que **não** está nos documentos que indexamos?
Vamos testar com uma pergunta sobre futebol (nossos documentos são sobre tecnologia/celulares).

In [19]:
pergunta_fora = "Quem ganhou a copa de 1994?"

print(f"🤖 Pergunta: {pergunta_fora}\n")
resposta_fora, docs_fora = rag_query(pergunta_fora, modelo='gpt-oss:latest')

print("📝 Resposta do Llama:")
print("-" * 40)
print(resposta_fora)
print("-" * 40)

print("\n📚 Documentos recuperados (provavelmente irrelevantes):")
for i, doc in enumerate(docs_fora):
    print(f"   {i+1}. {doc.page_content}")

🤖 Pergunta: Quem ganhou a copa de 1994?

📝 Resposta do Llama:
----------------------------------------
Desculpe, mas não tenho essa informação no contexto fornecido.
----------------------------------------

📚 Documentos recuperados (provavelmente irrelevantes):
   1. O atacante chutou a bola no ângulo e foi gol.
   2. Receita de lasanha à bolonhesa com muito queijo.
📝 Resposta do Llama:
----------------------------------------
Desculpe, mas não tenho essa informação no contexto fornecido.
----------------------------------------

📚 Documentos recuperados (provavelmente irrelevantes):
   1. O atacante chutou a bola no ângulo e foi gol.
   2. Receita de lasanha à bolonhesa com muito queijo.


### 💡 Análise do Erro (ou Sucesso)

Se o modelo respondeu "Não sei" ou deu uma resposta estranha, isso é **bom**!
Significa que ele está respeitando o contexto. Como não indexamos nada sobre futebol, ele não encontrou informações para responder.

**Lição:** O RAG depende 100% da qualidade da busca.
*   **Busca Ruim** (Garbage In) → **Resposta Ruim** (Garbage Out).

Por isso, gastamos tanto tempo nos labs anteriores aprendendo sobre Embeddings e Busca Vetorial! 🚀

### 🔍 Analisando o Resultado

Observe que o modelo não apenas "copiou" o texto, mas **interpretou** a informação.

- Você perguntou sobre "câmera boa"
- O documento falava sobre "lente periscópica incrível"
- O LLM conectou os pontos e recomendou o iPhone 15!

Isso é muito mais poderoso que uma busca simples por palavras-chave.

## 🎓 Resumo e Conceitos-Chave

### O que aprendemos

✅ **Persistência de índices** - Salvar e carregar FAISS do disco  
✅ **Ollama local** - Embeddings e LLM 100% gratuitos e privados  
✅ **Performance** - Carregar é 100x mais rápido que criar  
✅ **RAG completo** - Retrieval + Augmentation + Generation  
✅ **Privacidade** - Dados nunca saem da sua máquina

### Fluxo completo que implementamos

```text
1. Criar embeddings (via Ollama local)
   ↓
2. Criar índice FAISS (em RAM)
   ↓
3. Salvar no disco (.save_local) ← PERSISTÊNCIA!
   ↓
4. Carregar do disco (.load_local) ← REUTILIZAÇÃO!
   ↓
5. Fazer buscas normalmente
   ↓
6. Gerar respostas com Llama local
```

### Comparação: Ollama vs APIs de Nuvem

| Aspecto | Ollama Local | APIs Nuvem |
|---------|--------------|------------|
| **Custo** | $0.00 | $0.02-0.10 por 1M tokens |
| **Privacidade** | ✅ 100% local | ❌ Dados vão para nuvem |
| **Latência** | ~50-100ms | ~200-500ms |
| **Offline** | ✅ Funciona | ❌ Precisa internet |
| **Escalabilidade** | Limitada por hardware | Ilimitada |
| **Qualidade** | ⭐⭐⭐⭐ Excelente | ⭐⭐⭐⭐⭐ Melhor |

### Aplicações reais de RAG com Ollama

🤖 **Chatbots corporativos privados**
- Indexa documentação interna sensível
- LLM responde baseado nos docs da empresa
- Dados nunca saem do servidor

📚 **Assistentes de estudo offline**
- Indexa livros, apostilas, anotações
- Funciona sem internet
- Totalmente gratuito

🔧 **Suporte técnico local**
- Indexa manuais, FAQs, tickets antigos
- Resposta instantânea
- Sem custos de API

🏥 **Assistência médica privada**
- Indexa prontuários, estudos científicos
- 100% privado (LGPD/HIPAA compliant)
- LLM auxilia diagnósticos localmente

### 🧪 Experimentos para tentar

#### Experimento 1: Testar modelos diferentes
```python
# Embeddings menores (mais rápido)
embeddings_small = OllamaEmbeddings(
    model='all-minilm',  # 384 dims
    base_url=OLLAMA_BASE_URL
)

# Embeddings maiores (mais qualidade)
embeddings_large = OllamaEmbeddings(
    model='mxbai-embed-large',  # 1024 dims
    base_url=OLLAMA_BASE_URL
)

# Compare a qualidade das buscas!
```

In [ ]:
# Código aqui


#### Experimento 2: LLMs diferentes

```python
# Llama menor (mais rápido)
llm_fast = Ollama(model="llama3.2:1b", base_url=OLLAMA_BASE_URL)

# Llama maior (melhor qualidade)
llm_quality = Ollama(model="llama3.2:3b", base_url=OLLAMA_BASE_URL)

# Mistral (alternativa)
llm_mistral = Ollama(model="mistral:7b", base_url=OLLAMA_BASE_URL)

# Compare as respostas!
```

In [ ]:
# Código aqui


#### Experimento 3: Adicionar mais documentos
```python

novos_textos = [
    "Samsung Galaxy S23 tem câmera de 200MP",
    "Xiaomi Redmi Note 12 é um bom custo-benefício",
    "Google Pixel 8 Pro tem excelente fotografia noturna"
]

# Criar novo índice com todos os documentos
todos_textos = meus_textos + novos_textos
vector_store = FAISS.from_texts(todos_textos, embeddings)
vector_store.save_local(str(FAISS_PATH))

# Agora tem 8 documentos!
```

In [ ]:
# Código aqui


#### Experimento 4: RAG com streaming
```python
from langchain_community.llms import Ollama

def rag_stream(pergunta, k=2):
    """RAG com resposta em streaming (efeito ChatGPT)"""
    db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)
    docs = db.similarity_search(pergunta, k=k)
    contexto = "\n".join([f"- {doc.page_content}" for doc in docs])
    
    prompt = f"""Contexto:\n{contexto}\n\nPergunta: {pergunta}\n\nResposta:"""
    
    llm = Ollama(model="llama3.2:1b", base_url=OLLAMA_BASE_URL)
    
    # Streaming!
    for chunk in llm.stream(prompt):
        print(chunk, end='', flush=True)

# Testar
rag_stream("Qual é o melhor celular?")
```

In [ ]:
# Código aqui


### 💡 Boas práticas em produção com Ollama

1. **Versionamento de índices**
   ```python
   vector_store.save_local("indices/v1.0.0")
   vector_store.save_local("indices/v1.1.0")  # Após adicionar docs
   ```

2. **Monitoramento de recursos**
   ```python
   import psutil
   
   # CPU
   cpu_percent = psutil.cpu_percent(interval=1)
   print(f"CPU: {cpu_percent}%")
   
   # Memória
   mem = psutil.virtual_memory()
   print(f"RAM: {mem.percent}%")
   ```

3. **Metadados do índice**
   ```python
   import json
   from datetime import datetime
   
   metadata = {
       "created_at": datetime.now().isoformat(),
       "num_docs": len(meus_textos),
       "embedding_model": EMBEDDING_MODEL,
       "dimensions": 768,
       "llm_model": "llama3.2:1b"
   }
   
   with open(FAISS_PATH / "metadata.json", "w") as f:
       json.dump(metadata, f, indent=2)
   ```

4. **Cache de respostas**
   ```python
   from functools import lru_cache
   
   @lru_cache(maxsize=100)
   def rag_query_cached(pergunta, k=2):
       return rag_query(pergunta, k)
   
   # Perguntas repetidas são instantâneas!
   ```

5. **Health check do Ollama**
   ```python
   def check_ollama_health():
       try:
           response = requests.get(f'{OLLAMA_BASE_URL}/api/tags', timeout=5)
           return response.status_code == 200
       except:
           return False
   
   if not check_ollama_health():
       print("⚠️  Ollama não está respondendo!")
   ```

💡 **Dica final:** O RAG com Ollama é **ideal** para aplicações que precisam de **privacidade**, **controle de custos** e **funcionamento offline**. É a escolha perfeita para ambientes corporativos sensíveis ou aplicações que precisam rodar em edge devices!